In [63]:
import pandas as pd
import glob
import os
import numpy as np
from featureranker.utils import *
from featureranker.plots import *
from featureranker.rankers import *
from tqdm.auto import tqdm

xlsx_path = './betting_odds/'
txt_path = './game_data/'
paths = [path for path in glob.glob(xlsx_path + '*.xlsx')]
paths_txt = [path_txt for path_txt in glob.glob(txt_path + '*.txt')]
for path in paths:
    xlsx_dataframes = pd.read_excel(path)
    xlsx_dataframes.columns = xlsx_dataframes.columns.str.replace('[^a-zA-Z0-9]', '')
    view_data(xlsx_dataframes)


txt_dataframes = []
for path_txt in paths_txt:
    df = pd.read_csv(path_txt, delimiter = ",", quotechar='"', quoting=1, header = None)
    # df.columns = df.columns.astype(str).str.replace('[^a-zA-Z0-9]', '')
    txt_dataframes.append(df)
    view_data(df)
# txt_dataframes = txt_dataframes.replace({',': '', '\"': ''}, regex=True)

# If you want to view the file in Excel, uncomment the following line
# os.startfile(xlsx_files[0])


There are no NaN values in the dataset
The column 9th has 0.1% NaN values.
There are no NaN values in the dataset
There are no NaN values in the dataset
The column 9th has 0.1% NaN values.
There are no NaN values in the dataset
The column 9th has 0.1% NaN values.
The column Pitcher has 0.6% NaN values.
The column 9th has 0.1% NaN values.
The column Pitcher has 0.2% NaN values.
The column 8th has 0.1% NaN values.
The column 9th has 0.1% NaN values.
There are no NaN values in the dataset
There are no NaN values in the dataset
There are no NaN values in the dataset
The column 13 has 99.9% NaN values.
The column 14 has 100.0% NaN values.
The column 15 has 99.9% NaN values.
The column 85 has 100.0% NaN values.
The column 87 has 100.0% NaN values.
The column 97 has 50.5% NaN values.
The column 99 has 5.1% NaN values.
The column 159 has 99.5% NaN values.
The column 13 has 100.0% NaN values.
The column 14 has 100.0% NaN values.
The column 15 has 100.0% NaN values.
The column 81 has 0.3% NaN va

In [64]:
def process_dataframes(paths):
    xlsx_dataframes = [pd.read_excel(path) for path in paths]
    merged_df = pd.concat(xlsx_dataframes)
    home_teams_df = merged_df[merged_df['VH'] == 'H']
    visiting_teams_df = merged_df[merged_df['VH'] == 'V']

    selected_columns_df = pd.DataFrame()
    for i in range(len(xlsx_dataframes)):
        year = 2010 + i
        temp_df = pd.DataFrame({
            'date': home_teams_df['Date'].apply(lambda x: str(year) + str(x).zfill(4)).reset_index(drop=True),
            'home_team': home_teams_df['Team'].reset_index(drop=True),
            'visiting_team': visiting_teams_df['Team'].reset_index(drop=True),
            'home_open': home_teams_df['Open'].reset_index(drop=True),
            'visiting_open': visiting_teams_df['Open'].reset_index(drop=True)
        })
        selected_columns_df = pd.concat([selected_columns_df, temp_df])

    return selected_columns_df

print(process_dataframes(paths))


           date home_team visiting_team home_open visiting_open
0      20100404       BOS           NYY      -114          -106
1      20100405       WAS           PHI       170          -200
2      20100405       NYM           MIA      -115          -105
3      20100405       CIN           STL       135          -155
4      20100405       PIT           LOS       135          -155
...         ...       ...           ...       ...           ...
28001  20211027       HOU           ATL      -115          -105
28002  20211029       ATL           HOU      -115          -105
28003  20211030       ATL           HOU      -115          -105
28004  20211031       ATL           HOU      -105          -115
28005  20211102       HOU           ATL      -120           100

[336072 rows x 5 columns]


In [65]:
print(txt_dataframes)

[           0    1    2    3   4    5    6   7    8    9    ...  \
0     20100404    0  Sun  NYA  AL    1  BOS  AL    1    7  ...   
1     20100405    0  Mon  MIN  AL    1  ANA  AL    1    3  ...   
2     20100405    0  Mon  CLE  AL    1  CHA  AL    1    0  ...   
3     20100405    0  Mon  DET  AL    1  KCA  AL    1    8  ...   
4     20100405    0  Mon  SEA  AL    1  OAK  AL    1    5  ...   
...        ...  ...  ...  ...  ..  ...  ...  ..  ...  ...  ...   
2425  20101003    0  Sun  CHN  NL  162  HOU  NL  162    0  ...   
2426  20101003    0  Sun  ARI  NL  162  LAN  NL  162    1  ...   
2427  20101003    0  Sun  WAS  NL  162  NYN  NL  162    2  ...   
2428  20101003    0  Sun  SDN  NL  162  SFN  NL  162    0  ...   
2429  20101003    0  Sun  COL  NL  162  SLN  NL  162    1  ...   

                      151  152       153             154  155       156  \
0               J.D. Drew    9  camem001    Mike Cameron    8  scutm001   
1          Howie Kendrick    4  woodb003    Brandon Wood

In [66]:
def extract_data(txt_dataframes):
    extracted_data_df = pd.DataFrame()
    for txt_df in txt_dataframes:
        # Extracting required data from the columns based on the provided indices
        temp_df = pd.DataFrame({
            'date': txt_df.iloc[:, 0].apply(lambda x: str(x)),
            'home_team': txt_df.iloc[:, 6],
            'visiting_team': txt_df.iloc[:, 3],
        })
        # Extracting player names for home and visiting teams
        home_players = [txt_df.iloc[:, 133+3*i].str.split(',').str[0].replace(np.nan, 'Unknown Player') for i in range(9)]
        visiting_players = [txt_df.iloc[:, 106+3*i].str.split(',').str[0].replace(np.nan, 'Unknown Player') for i in range(9)]
        for i in range(9):
            temp_df['home_player_'+str(i+1)] = home_players[i]
            temp_df['visiting_player_'+str(i+1)] = visiting_players[i]
        extracted_data_df = pd.concat([extracted_data_df, temp_df])

    return extracted_data_df

final_df = extract_data(txt_dataframes)
print(final_df)








          date home_team visiting_team    home_player_1 visiting_player_1  \
0     20100404       BOS           NYA  Jacoby Ellsbury       Derek Jeter   
1     20100405       ANA           MIN      Erick Aybar       Denard Span   
2     20100405       CHA           CLE      Juan Pierre  Asdrubal Cabrera   
3     20100405       KCA           DET    David DeJesus    Austin Jackson   
4     20100405       OAK           SEA      Rajai Davis     Ichiro Suzuki   
...        ...       ...           ...              ...               ...   
2425  20221005       CLE           KCA      Steven Kwan       MJ Melendez   
2426  20221005       HOU           PHI      Kyle Tucker    Kyle Schwarber   
2427  20221005       OAK           ANA        Tony Kemp      Luis Rengifo   
2428  20221005       SEA           DET  Julio Rodriguez       Akil Baddoo   
2429  20221005       TEX           NYA    Marcus Semien       DJ LeMahieu   

        home_player_2 visiting_player_2    home_player_3   visiting_player_

In [67]:
print(final_df.iloc[0,1])


BOS


In [68]:
print(selected_columns_df)

NameError: name 'selected_columns_df' is not defined